In [1]:
import numpy as np
import pandas as pd
import csv
from scamp import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Softmax,Input
import keras.backend as K 

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Species.csv', sep=",",index_col = 0)
print(df)

      Cantus  Counter
0         58       67
1         66       69
2         64       71
3         68       76
4         66       74
...      ...      ...
9995      64       72
9996      56       60
9997      57       61
9998      61       65
9999      56       64

[10000 rows x 2 columns]


In [3]:
n = df.shape[0]
n

10000

In [4]:
X = tf.Variable(np.zeros((2,n*2)),dtype='int32')  # any data tensor
for i in range(0,2*n,2):
    X[0,i].assign(df['Cantus'][i/2] - 43)
    X[1,i].assign(0)
    X[0,i+1].assign(df['Counter'][i/2] - 55)
    X[1,i+1].assign(1)
X

<tf.Variable 'Variable:0' shape=(2, 20000) dtype=int32, numpy=
array([[15, 12, 23, ..., 10, 13,  9],
       [ 0,  1,  0, ...,  1,  0,  1]])>

In [5]:
depth = 30
X = tf.concat([tf.one_hot(X[0,:], depth),tf.dtypes.cast(tf.reshape(X[1,:],[20000,1]),tf.float32)],1)
X

<tf.Tensor: shape=(20000, 31), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [6]:
train_dataset = tf.data.Dataset.from_tensor_slices(X[:16000,:])
valid_dataset = tf.data.Dataset.from_tensor_slices(X[16000:19000,:])
test_dataset = tf.data.Dataset.from_tensor_slices(X[19000:,:])

In [7]:
n_steps = 20
batch_size = 18
window_length = n_steps
dataset = test_dataset.window(window_length, shift=2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
shuffle_dataset = dataset.shuffle(20000).batch(batch_size,drop_remainder=True)

In [8]:
def creat_model(batch_size,Type):
    
    rnn_units = 12
    n_notes = 30

    input_A = keras.layers.Input(shape=(None,31), name="input_A") # (15, 20, 31)
    input_B = keras.layers.Input(shape=(1,31), name="input_B") # (15, 1, 31)

    hidden1 = keras.layers.LSTM(rnn_units, return_sequences=True)(input_A) # (15,20,12)
    hidden2 = keras.layers.LSTM(rnn_units, return_sequences=True)(hidden1) #(15,20,12)
    
    output_RNN = keras.layers.Dense(n_notes, activation='softmax', name = 'output_RNN')(hidden2) #(15,20,30)

    e = keras.layers.Dense(1, activation='tanh')(hidden2) #(15,20,1)
    e = keras.layers.Reshape([-1])(e) #(15,20)

    alpha = keras.layers.Activation('softmax')(e) #(15,20)
    c = keras.layers.Permute([2, 1])(keras.layers.RepeatVector(rnn_units)(alpha)) #(15,20,12)
    c = keras.layers.Multiply()([hidden2, c]) #(15,20,12)
    c = keras.layers.Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c) #(15,12)

    output_A = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_A')(c) #(15,30) (0, cantus)
    reshape = tf.reshape(output_A,[-1,1,30]) # (15, 1, 30)
    if Type == "_0_1":
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        input_output_A = tf.concat((reshape,zero),2) # (15, 1, 31)  (0, cantus)
    elif Type == "_1_0":
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        input_output_A = tf.concat((reshape,one),2) # (15, 1, 31)  (0, cantus)

    aux1 = keras.layers.SimpleRNN(rnn_units)(input_B, initial_state=c) #(15,12)
    aux2 = keras.layers.SimpleRNN(rnn_units)(input_output_A, initial_state=c) #(15,12)
    output_B1 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B1')(aux1) #(15,30) (1, counter)
    output_B2 = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B2')(aux2) #(15,30) (1, counter)

    model = keras.models.Model([input_A, input_B], [output_RNN, output_A, output_B1, output_B2])
    
    aux_model1 = keras.models.Model([input_A, input_B], output_B1)
    aux_model2 = keras.models.Model(input_A, [output_A, output_B2])
    
    sub_model1 = keras.models.Model(input_A, output_A)
    sub_model2 = keras.models.Model(input_A, output_B2)
    
    att_model = keras.models.Model([input_A, input_B], alpha)
    
    return model,aux_model1,aux_model2,sub_model1,sub_model2,att_model

In [9]:
model_0_1_,aux_model1_0_1_,aux_model2_0_1_, sub_model1_0_1_,sub_model2_0_1_,att_model_0_1_ = creat_model(batch_size,'_0_1')
model_1_0_,aux_model1_1_0_,aux_model2_1_0_, sub_model1_1_0_,sub_model2_1_0_,att_model_1_0_ = creat_model(batch_size,'_1_0')

In [10]:
model_0_1_.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_A (InputLayer)            [(None, None, 31)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, None, 12)     2112        input_A[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, None, 12)     1200        lstm[0][0]                       
__________________________________________________________________________________________________
dense (Dense)                   (None, None, 1)      13          lstm_1[0][0]                     
_______________________________________________________________________________________

In [11]:
model_0_1_.load_weights('model_0_1_weight.h5')
model_1_0_.load_weights('model_1_0_weight.h5')

In [12]:
def generate_sequence(initial,seq_length,mode,generate_mode):
    """
    Arguments:
    initial -- initial sequence of shape (1,20,31) (1 sample, sequence of length 20, vector fearure: one-hot(30);cantus(0),counter(1))
    seq_length -- # pairs of notes generated
    
    """
    ### START CODE HERE ###
    sequence = initial
    pred_0_1_0 = []
    pred_0_1_1 = []
    pred2_0_1_1 = []
    pred_1_0_1 = []
    pred_1_0_0 = []
    pred2_1_0_0 = []
    
    for i in range(seq_length):
        
        [pred_o,a] = aux_model2_0_1_.predict(initial) #(18,30)
        pred_o = pred_o.reshape(30,)
        a = a.reshape(30,)
        
        pred_o_max = np.where(pred_o == np.max(pred_o))[0][0]
        pred_o_random = np.random.choice(30, 1, p=pred_o)[0]
        
        a_max = np.where(a == np.max(a))[0][0]
        a_random = np.random.choice(30, 1, p=a)[0]
                

        [pred_a,o] = aux_model2_1_0_.predict(initial) #(18,30)
        pred_a = pred_a.reshape(30,)
        o = o.reshape(30,)
        
        pred_a_max = np.where(pred_a == np.max(pred_a))[0][0]
        pred_a_random = np.random.choice(30, 1, p=pred_a)[0]
        
        o_max = np.where(o == np.max(o))[0][0]
        o_random = np.random.choice(30, 1, p=o)[0]
        
        
        if generate_mode == "max":
            pred_obv = pred_o_max # pred_o_random
            action = a_max # a_random
            pred_action = pred_a_max # pred_a_random
            observation = o_max # o_random
        elif generate_mode == "random":
            pred_obv = pred_o_random # pred_o_random
            action = a_random # a_random
            pred_action = pred_a_random # pred_a_random
            observation = o_random # o_random
            
            
        pred_0_1_0.append(pred_obv+43)
        pred_0_1_1.append(action+55)
        pred_1_0_1.append(pred_action+55)
        pred_1_0_0.append(observation+43)
        
        
        one_hot = tf.reshape(tf.one_hot(pred_obv, 30),[1,1,30])
        out_0_1_0 = tf.concat((one_hot,[[[0]]]),2)
        
        one_hot = tf.reshape(tf.one_hot(action, 30),[1,1,30])
        out_0_1_1 = tf.concat((one_hot,[[[1]]]),2)
        
        one_hot = tf.reshape(tf.one_hot(pred_action, 30),[1,1,30])
        out_1_0_1 = tf.concat((one_hot,[[[1]]]),2)
        
        one_hot = tf.reshape(tf.one_hot(observation, 30),[1,1,30])
        out_1_0_0 = tf.concat((one_hot,[[[0]]]),2)
        
        a2 = aux_model1_0_1_.predict([initial,out_0_1_0])
        o2 = aux_model1_1_0_.predict([initial,out_1_0_1])
        
        a2 = a2.reshape(30,)
        a2_max = np.where(a2 == np.max(a2))[0][0]
        a2_random = np.random.choice(30, 1, p=a2)[0]
        
        o2 = o2.reshape(30,)
        o2_max = np.where(o2 == np.max(o2))[0][0]
        o2_random = np.random.choice(30, 1, p=o2)[0]
        
        if generate_mode == "max":
            action2 = a2_max # a_random
            observation2 = o2_max # o_random
        elif generate_mode == "random":
            action2 = a2_random # a_random
            observation2 = o2_random # o_random
            
        pred2_0_1_1.append(action2+55)
        pred2_1_0_0.append(observation2+43)
        
        one_hot = tf.reshape(tf.one_hot(action2, 30),[1,1,30])
        out2_0_1_1 = tf.concat((one_hot,[[[1]]]),2)
        
        one_hot = tf.reshape(tf.one_hot(observation2, 30),[1,1,30])
        out2_1_0_0 = tf.concat((one_hot,[[[0]]]),2)
            
        if mode == "first meet":
            sequence = tf.concat((sequence,out_1_0_0),1)
            sequence = tf.concat((sequence,out_0_1_1),1)
        elif mode == "brain 1":
            sequence = tf.concat((sequence,out_0_1_0),1)
            sequence = tf.concat((sequence,out_0_1_1),1)
        elif mode == "brain 2":
            sequence = tf.concat((sequence,out_1_0_0),1)
            sequence = tf.concat((sequence,out_1_0_1),1)
        elif mode == "brain 1_certainty":
            sequence = tf.concat((sequence,out_0_1_0),1)
            sequence = tf.concat((sequence,out2_0_1_1),1)
        elif mode == "brain 2_certainty":
            sequence = tf.concat((sequence,out2_1_0_0),1)
            sequence = tf.concat((sequence,out_1_0_1),1)

        initial = sequence[:,-20:,:]
        
    return sequence,pred_0_1_0,pred_0_1_1,pred_1_0_1,pred_1_0_0,pred2_0_1_1,pred2_1_0_0

In [13]:
seq_length = 10

#generate_mode = "max"
generate_mode = "random"


mode = "first meet"
mode = "brain 1"
mode = "brain 2"
mode = "brain 1_inside"
mode = "brain 2_inside"


for window in shuffle_dataset.take(1):
    sequence = window
    initial = window
    
    pred_0_1_1 = np.zeros((batch_size,seq_length))
    pred_0_1_0 = np.zeros((batch_size,seq_length))
    pred2_0_1_1 = np.zeros((batch_size,seq_length))
    
    pred_1_0_0 = np.zeros((batch_size,seq_length))
    pred_1_0_1 = np.zeros((batch_size,seq_length))
    pred2_1_0_0 = np.zeros((batch_size,seq_length))
    
    obv_weight = np.zeros((batch_size,seq_length))
    act_weight = np.zeros((batch_size,seq_length))

    for s in range(seq_length):

        [pred_o,a] = aux_model2_0_1_.predict(initial) #(18,30)

        for i in range(batch_size):
            if generate_mode == "max":
                pred_0_1_0[i,s] = np.where(pred_o[i,:] == np.max(pred_o[i,:]))[0][0]
                pred_0_1_1[i,s] = np.where(a[i,:] == np.max(a[i,:]))[0][0]
            elif generate_mode == "random":
                pred_0_1_0[i,s] = np.random.choice(30, 1, p=pred_o[i,:])[0]
                pred_0_1_1[i,s] = np.random.choice(30, 1, p=a[i,:])[0]


        [pred_a,o] = aux_model2_1_0_.predict(initial) #(18,30)
        
        for i in range(batch_size):
            if generate_mode == "max":
                pred_1_0_1[i,s] = np.where(pred_a[i,:] == np.max(pred_a[i,:]))[0][0]
                pred_1_0_0[i,s] = np.where(o[i,:] == np.max(o[i,:]))[0][0]
            elif generate_mode == "random":
                pred_1_0_1[i,s] = np.random.choice(30, 1, p=pred_a[i,:])[0]
                pred_1_0_0[i,s] = np.random.choice(30, 1, p=o[i,:])[0]
            
            index = int(pred_1_0_0[i,s])
            obv_weight[i,s] = pred_o[i,index]
            
            index = int(pred_0_1_1[i,s])
            act_weight[i,s] = pred_a[i,index]
            
        
        one_hot = tf.one_hot(pred_0_1_0[:,s], 30) #(18,30)
        reshape = tf.reshape(one_hot,[batch_size,1,30])
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        out_0_1_0 = tf.concat((reshape,zero),2) # (18, 1, 31)  (1, counter)
        
        one_hot = tf.one_hot(pred_0_1_1[:,s], 30) #(18,30)
        reshape = tf.reshape(one_hot,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        out_0_1_1 = tf.concat((reshape,one),2) # (18, 1, 31)  (1, counter)
        
        one_hot = tf.one_hot(pred_1_0_1[:,s], 30) #(18,30)
        reshape = tf.reshape(one_hot,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        out_1_0_1 = tf.concat((reshape,one),2) # (18, 1, 31)  (1, counter)
        
        one_hot = tf.one_hot(pred_1_0_0[:,s], 30) #(18,30)
        reshape = tf.reshape(one_hot,[batch_size,1,30])
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        out_1_0_0 = tf.concat((reshape,zero),2) # (18, 1, 31)  (1, counter)

        
        a2 = aux_model1_0_1_.predict([initial,out_0_1_0])
        o2 = aux_model1_1_0_.predict([initial,out_1_0_1])
        
        for i in range(batch_size):
            if generate_mode == "max":
                pred2_0_1_1[i,s] = np.where(a2[i,:] == np.max(a2[i,:]))[0][0]
                pred2_1_0_0[i,s] = np.where(o2[i,:] == np.max(o2[i,:]))[0][0]
            elif generate_mode == "random":
                pred2_0_1_1[i,s] = np.random.choice(30, 1, p=a2[i,:])[0]
                pred2_1_0_0[i,s] = np.random.choice(30, 1, p=o2[i,:])[0]

        one_hot = tf.one_hot(pred2_0_1_1[:,s], 30) #(18,30)
        reshape = tf.reshape(one_hot,[batch_size,1,30])
        one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
        out2_0_1_1 = tf.concat((reshape,one),2) # (18, 1, 31)  (1, counter)
        
        one_hot = tf.one_hot(pred2_1_0_0[:,s], 30) #(18,30)
        reshape = tf.reshape(one_hot,[batch_size,1,30])
        zero = tf.reshape(tf.zeros(batch_size),[batch_size,1,1])
        out2_1_0_0 = tf.concat((reshape,zero),2) # (18, 1, 31)  (1, counter)
        

        if mode == "first meet":
            sequence = tf.concat((sequence,out_1_0_0),1)
            sequence = tf.concat((sequence,out_0_1_1),1)
        elif mode == "brain 1":
            sequence = tf.concat((sequence,out_0_1_0),1)
            sequence = tf.concat((sequence,out2_0_1_1),1)
        elif mode == "brain 2":
            sequence = tf.concat((sequence,out2_1_0_0),1)
            sequence = tf.concat((sequence,out_1_0_1),1)
        elif mode == "brain 1_inside":
            sequence = tf.concat((sequence,out_0_1_0),1)
            sequence = tf.concat((sequence,out_0_1_1),1)
        elif mode == "brain 2_inside":
            sequence = tf.concat((sequence,out_1_0_0),1)
            sequence = tf.concat((sequence,out_1_0_1),1)

            
        initial = sequence[:,-20:,:]
        
    print('collective_action:',pred_0_1_1)
    print('collective_observation:',pred_1_0_0)
    print('')
    print('musician1_action:',pred2_0_1_1)
    print('musician1_observation:',pred_0_1_0)
    print('')
    print('musician2_action:',pred_1_0_1)
    print('musician2_observation:',pred2_1_0_0)
    print('')
    print('musician1_action_pred:',pred_0_1_1)
    print('musician1_observation_pred:',pred_0_1_0)
    print('')
    print('musician2_action_pred:',pred_1_0_1)
    print('musician2_observation_pred:',pred_1_0_0)
    print('')
    
    step = [];
    for j in range(seq_length):
        step.append("step" + str(j))
    step.append("sequence match")
        
    sample = [];
    for i in range(batch_size):
        sample.append("sequence" + str(i))
    sample.append("step match")
    
    table = [];
    for i in range(batch_size):
        seq = [];
        for j in range(seq_length):
            seq.append(str(int(pred_0_1_1[i,j])) + "/(" + '%.2f'%(act_weight[i,j]) + ")")
        match_a = np.sum(act_weight[i,:])/seq_length
        seq.append('%.2f'%(match_a))
        table.append(seq)
        
    last_row = [];
    for j in range(seq_length):
        match_a = np.sum(act_weight[:,j])/batch_size
        last_row.append('%.2f'%(match_a))
    total_a = np.sum(act_weight)/(seq_length*batch_size)
    last_row.append('%.2f'%(total_a))
    table.append(last_row)
    
    format_row = "{:>11}" * (seq_length + 2)
    print(format_row.format("act/(pred_act_weight)", *step))
    for team, row in zip(sample, table):
        print(format_row.format(team, *row))

    print('')
        
    table2 = [];
    for i in range(batch_size):
        seq = [];
        for j in range(seq_length):
            seq.append(str(int(pred_1_0_0[i,j])) + "/(" + '%.2f'%(obv_weight[i,j]) + ")")
        match_o = np.sum(obv_weight[i,:])/seq_length
        seq.append('%.2f'%(match_o))
        table2.append(seq)
        
    last_row = [];
    for j in range(seq_length):
        match_o = np.sum(obv_weight[:,j])/batch_size
        last_row.append('%.2f'%(match_o))
    total_o = np.sum(obv_weight)/(seq_length*batch_size)
    last_row.append('%.2f'%(total_o))
    table2.append(last_row)
    
    format_row = "{:>11}" * (seq_length + 2)
    print(format_row.format("obv/(pred_obv_weight)", *step))
    for team, row in zip(sample, table2):
        print(format_row.format(team, *row))

collective_action: [[23. 11. 23. 19. 21. 11. 21. 17. 25. 17.]
 [ 7.  7. 10.  1.  9.  2.  3.  7.  7.  6.]
 [ 0.  7. 17. 14. 13. 13. 17. 18. 18.  8.]
 [20. 16. 26. 25. 19. 14. 17. 15. 17. 12.]
 [ 4.  8. 13.  3.  6. 16. 16.  6.  0. 10.]
 [ 6. 17. 13.  9. 17.  6. 15. 23. 17. 10.]
 [10. 15. 14. 11. 13.  8. 16. 15.  9.  9.]
 [ 3.  8. 12.  4.  4. 16. 17. 10. 11. 13.]
 [ 4.  7.  5. 10.  7. 13.  1. 10.  9.  6.]
 [ 5.  8.  4.  8.  5.  5.  2.  7.  6.  6.]
 [18. 18. 12. 15. 12. 20. 13. 12. 13. 20.]
 [11. 12. 12. 18. 17. 21. 11. 15.  7.  8.]
 [13. 17. 21. 19. 16. 15. 24. 15. 20. 21.]
 [24. 22. 20.  9. 12. 15. 14.  9.  7. 10.]
 [17. 18. 20. 19. 18. 17. 24. 14. 13. 16.]
 [21. 17. 11.  7. 13. 17. 16. 11.  6.  3.]
 [ 5.  4.  7. 11.  7.  0. 11.  9.  6. 14.]
 [20. 25. 17. 17. 23. 25. 16. 23. 23. 28.]]
collective_observation: [[23. 20. 20. 26. 23. 25. 25. 27. 27. 29.]
 [15. 12. 13. 11.  9. 11. 11. 15. 17. 20.]
 [12. 24. 18. 21. 20. 29. 22. 20. 18.  8.]
 [28. 29. 27. 29. 28. 27. 27. 22. 20. 29.]
 [11. 11. 

In [24]:
format_row = "{:>11}" * (seq_length + 2)
print(format_row.format("obv/(pred_obv_weight)", *step))
for team, row in zip(sample, table):
    print(format_row.format(team, *row))

obv/(pred_obv_weight)      step0      step1      step2      step3      step4      step5      step6      step7      step8      step9sequence match
  sequence0   5/(0.10)   6/(0.09)   1/(0.08)   7/(0.08)   7/(0.09)   3/(0.09)   1/(0.05)   8/(0.06)   9/(0.08)   8/(0.05)       0.08
  sequence1  14/(0.03)  20/(0.09)  16/(0.05)  16/(0.07)  19/(0.07)  18/(0.10)  25/(0.02)  18/(0.09)  16/(0.08)  21/(0.10)       0.07
  sequence2  15/(0.11)  16/(0.08)  15/(0.05)  11/(0.09)  12/(0.11)  11/(0.09)   9/(0.09)   7/(0.11)   5/(0.12)   8/(0.08)       0.09
  sequence3  18/(0.10)  13/(0.05)  20/(0.12)  17/(0.10)  15/(0.08)  20/(0.08)  10/(0.04)  10/(0.08)  10/(0.07)   9/(0.08)       0.08
  sequence4  21/(0.06)  14/(0.05)  15/(0.04)  22/(0.10)  21/(0.11)  18/(0.09)  12/(0.07)   9/(0.03)  15/(0.10)  19/(0.06)       0.07
  sequence5   5/(0.10)  10/(0.04)   9/(0.08)   5/(0.11)   0/(0.07)   6/(0.08)   8/(0.06)  10/(0.10)  13/(0.07)   6/(0.07)       0.08
  sequence6   8/(0.08)  11/(0.09)  10/(0.10)   4/(0.06) 

In [13]:
X_start_A = tf.reshape(X[19040:19060,:],[1,20,31])
seq_length = 10
sequence,pred_0_1_0,pred_0_1_1,pred_1_0_1,pred_1_0_0,pred2_0_1_1,pred2_1_0_0 = generate_sequence(X_start_A,seq_length,"first meet","max")
sequence

<tf.Tensor: shape=(1, 40, 31), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 1.]]], dtype=float32)>

In [14]:
sequence = tf.reshape(sequence,[-1,31])
sequence = sequence[-20:,:]
row,col = sequence.shape
cantus_lst = [];
counter_lst = [];
for i in range(row):
    clas = np.where(sequence[i,:-1] == 1)[0][0]
    if sequence[i,-1] == 0:
        cantus_lst.append(clas+43)
    elif sequence[i,-1] == 1:
        counter_lst.append(clas+55)

In [15]:
cantus_lst

[65, 58, 55, 52, 52, 52, 52, 52, 52, 54]

In [16]:
counter_lst

[55, 55, 55, 55, 55, 55, 55, 55, 55, 55]

In [17]:
np.array(counter_lst) - np.array(cantus_lst) # real harmonic

array([-10,  -3,   0,   3,   3,   3,   3,   3,   3,   1], dtype=int64)

In [ ]:
s = Session()
s.tempo = 180
piano1 = s.new_part("piano")
piano2 = s.new_part("piano")

def cantus():
    for i in cantus_lst:
        piano2.play_note(i,1,4)
        
def counter():
    for i in counter_lst:
        piano1.play_note(i,1,4)
        
        
s.fast_forward_to_beat(100)

s.start_transcribing()
s.fork(counter)
s.fork(cantus)
s.wait_for_children_to_finish()
performance = s.stop_transcribing()
performance.to_score(title = "First Species Counterpoint", composer = "My programme",time_signature = "4/4").show_xml()

Using preset Piano Merlin for piano


ERROR:root:name 'counter_lst' is not defined
Traceback (most recent call last):
  File "C:\Users\79244\AppData\Roaming\Python\Python37\site-packages\clockblocks\clock.py", line 846, in _process
    process_function(*args, **kwds)
  File "<ipython-input-2-ab161ce0ff4b>", line 11, in counter
    for i in counter_lst:
NameError: name 'counter_lst' is not defined


Using preset Piano Merlin for piano
